In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Configurações do ativo e parâmetros
ticker = "WINM24"  # Atualize para o contrato vigente
lote = 1
stop_loss = 200  # Em pontos
take_profit = 400  # Em pontos

# Conectar ao MetaTrader 5
if not mt5.initialize():
    print("Falha ao conectar ao MT5")
    exit()

def obter_dados():
    """ Obtém os últimos 200 candles do Mini Índice """
    rates = mt5.copy_rates_from_pos(ticker, mt5.TIMEFRAME_M5, 0, 200)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

def calcular_indicadores(df):
    """ Calcula indicadores técnicos """
    df['mm9'] = df['close'].rolling(9).mean()
    df['mm21'] = df['close'].rolling(21).mean()
    df['rsi'] = 100 - (100 / (1 + (df['close'].diff().apply(lambda x: max(x, 0)).rolling(14).mean() /
                                  df['close'].diff().apply(lambda x: abs(x)).rolling(14).mean())))
    df['volatilidade'] = df['close'].rolling(21).std()
    df.dropna(inplace=True)
    return df

def preparar_dados(df):
    """ Prepara os dados para treino e previsão """
    df['target'] = np.where(df['close'].shift(-1) > df['close'], 1, 0)
    X = df[['mm9', 'mm21', 'rsi', 'volatilidade']]
    y = df['target']
    return X, y

def treinar_modelo(X, y):
    """ Treina o modelo de Machine Learning """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    modelo = RandomForestClassifier(n_estimators=100, random_state=42)
    modelo.fit(X_scaled, y)
    return modelo, scaler

def prever_sinal(modelo, scaler, df):
    """ Faz previsão da próxima movimentação """
    X_novo = df[['mm9', 'mm21', 'rsi', 'volatilidade']].iloc[-1:].values
    X_novo_scaled = scaler.transform(X_novo)
    previsao = modelo.predict(X_novo_scaled)[0]
    return 'compra' if previsao == 1 else 'venda'

def enviar_ordem(tipo):
    """ Envia ordem para o MetaTrader 5 """
    price = mt5.symbol_info_tick(ticker).ask if tipo == 'compra' else mt5.symbol_info_tick(ticker).bid
    sl = price - stop_loss if tipo == 'compra' else price + stop_loss
    tp = price + take_profit if tipo == 'compra' else price - take_profit
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": lote,
        "type": mt5.ORDER_TYPE_BUY if tipo == 'compra' else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": sl,
        "tp": tp,
        "deviation": 10,
        "magic": 123456,
        "comment": "Bot IA",
        "type_filling": mt5.ORDER_FILLING_IOC,
        "type_time": mt5.ORDER_TIME_GTC,
    }
    result = mt5.order_send(request)
    print(f"Ordem enviada: {tipo}, Resultado: {result}")

# Coleta dados e treina o modelo
df = obter_dados()
df = calcular_indicadores(df)
X, y = preparar_dados(df)
modelo, scaler = treinar_modelo(X, y)

# Loop do bot
while True:
    df = obter_dados()
    df = calcular_indicadores(df)
    sinal = prever_sinal(modelo, scaler, df)
    
    if sinal in ['compra', 'venda']:
        enviar_ordem(sinal)
    
    time.sleep(300)  # Espera 5 minutos antes de rodar novamente

mt5.shutdown()

KeyError: '01/01/2025'